In [20]:
# !pip install --upgrade pip
# !pip install contractions
# !pip install --upgrade azure-cognitiveservices-language-textanalytics

In [21]:
import json
import nltk
import keys
import requests
import contractions
import pandas as pd

from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient

#from keys import subscription_key, text_analytics_base_url, bing_spell_check_key, spell_check_url 

In [23]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
sample_tweets = pd.DataFrame.from_csv(
    "sample_tweets.csv",
    index_col=None
)

/data/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  app.launch_new_instance()


In [25]:
sample_tweets["id"] = sample_tweets.index
sample_tweets.rename(columns={"Body text": "text"}, inplace=True)
#sample_tweets.text = sample_tweets.text.str.lower()
sample_tweets.head(2)

,Down ID,Posted time,text,Point latitiude,Point longitude,Bio location,Image url,id
0,"""tag:search.twitter.com,2005:1109081183105835008""",22/03/2019 13:15,"""Primeape was recently spotted trying to make ...",NaN,NaN,NaN,"""https://pbs.twimg.com/profile_images/76017773...",0
1,"""tag:search.twitter.com,2005:1109080961277407234""",22/03/2019 13:14,"""@gianninewbon Meet me at London Bridge at hal...",NaN,NaN,"""Derbados""","""https://pbs.twimg.com/profile_images/10569615...",1


In [26]:
sample_tweets.count()

Down ID            44816
Posted time        44816
text               44816
Point latitiude      701
Point longitude      701
Bio location       31946
Image url          42978
id                 44816
dtype: int64

In [27]:
def spell_check(sentence):
    data = {'text': sentence}
    params = {
        'mkt':'en-us',
        'mode':'proof'
        }
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Ocp-Apim-Subscription-Key': bing_spell_check_key,
        }
    response = requests.post(spell_check_url, headers=headers, params=params, data=data)
    json_response = response.json()

    for token in json_response["flaggedTokens"]:
        sentence = sentence.replace(str(token["token"]), token["suggestions"][0]["suggestion"])
    
    return sentence

### Key Phrases

In [31]:
# subset_tweets = sample_tweets[["id", "text"]].sample(1000, random_state=1)
# stop_words = set(stopwords.words('english'))
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
counter = 0
def process_tweet(tweet):
    global counter
    counter += 1
    if counter%1000 == 0:
        print("Processing {} of {}".format(counter, len(sample_tweets)))
        print(datetime.now())

    try:
        normalized_tweet = tweet.strip()
        if len(normalized_tweet) > 0:
            normalized_tweet = spell_check(str(tweet)).lower().strip()
            normalized_tweet = contractions.fix(normalized_tweet)
            tokenized = tokenizer.tokenize(normalized_tweet)
            # tokenized = nltk.word_tokenize(normalized_tweet)
            # non_stop_words = [word for word in tokenized if (word not in stop_words)]
            isascii = lambda s: len(s) == len(s.encode())
            ascii_words = [word for word in tokenized[1:-1] if isascii(word)]
            normalized_tweet = " ".join(ascii_words)
            return normalized_tweet
        else:
            return None
    except Exception as error_msg:
        print("**********************")
        print("Tweet is {}".format(tweet))
        print(error_msg)
        return None
        

In [ ]:
print("Process started at {}".format(datetime.now()))
sample_tweets["processed_text"] = sample_tweets["text"].apply(process_tweet)
print("Process ended at {}".format(datetime.now()))

Process started at 2019-06-25 15:26:08.478334


In [ ]:
len(sample_tweets)

In [ ]:
sample_tweets.head()

In [ ]:
sample_tweets.dropna(inplace=True, subset = ["processed_text"])

In [ ]:
len(sample_tweets)

In [ ]:
documents_str = sample_tweets.to_json(orient = "records")
documents_json = json.loads(json.dumps({"documents" : json.loads(documents_str)}))

In [ ]:
sample_tweets.head()

In [ ]:
documents_json["documents"][0:5]

In [ ]:
len(documents_json["documents"])

In [ ]:
keyphrase_url = text_analytics_base_url + "keyPhrases"

headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(keyphrase_url, headers=headers, json=documents_json)
key_phrases = response.json()

key_phrases_df = pd.DataFrame.from_dict(key_phrases['documents'])
key_phrases_df.head()

In [ ]:
len(key_phrases_df)

In [ ]:
(
  key_phrases_df["keyPhrases"].apply(lambda x: len(x)).mean(),
  key_phrases_df["keyPhrases"].apply(lambda x: len(x)).max(),
  key_phrases_df["keyPhrases"].apply(lambda x: len(x)).min()
)

In [ ]:
def no_key_phrases(key_phrases):
    if len(key_phrases) == 0:
        return 1
not_covered =  key_phrases_df["keyPhrases"].apply(no_key_phrases).sum()

print("Number of posts not covered by Key Phrases: {} or {} %".format(
    not_covered, 
    (not_covered/len(key_phrases_df)*100)
    )
)

In [ ]:
def drop_long_key_phrases(keyPhrases):
    for phrase in keyPhrases:
        if len(phrase.split(" ")) >  5:
            keyPhrases.remove(phrase)
    return keyPhrases

In [ ]:
key_phrases_df["shortKeyPhrases"] = key_phrases_df["keyPhrases"].apply(drop_long_key_phrases)

In [ ]:
(
  key_phrases_df["shortKeyPhrases"].apply(lambda x: len(x)).mean(),
  key_phrases_df["shortKeyPhrases"].apply(lambda x: len(x)).max(),
  key_phrases_df["shortKeyPhrases"].apply(lambda x: len(x)).min()
)

In [ ]:
key_phrases_df[key_phrases_df["shortKeyPhrases"].map(len)>5].values

In [ ]:
def no_key_phrases(row):
    if len(row.shortKeyPhrases) == 0:
        print(row.id)
        return 1
not_covered =  key_phrases_df.apply(no_key_phrases, axis=1).sum()

print("Number of posts not covered by Shortened Key Phrases: {} or {} %".format(
    not_covered, 
    (not_covered/len(key_phrases_df)*100)
    )
)

In [ ]:
# sample_tweets[sample_tweets.id==28909].text.values

In [ ]:
unique_key_phrases = set(sum(key_phrases_df["shortKeyPhrases"].values, []))

key_phrases_category = {}
for key in unique_key_phrases:
    key_phrases_category[key] = []

def map_categories(row):
    for phrase in row["shortKeyPhrases"]:
        key_phrases_category[phrase].append(row["id"])
    return None

key_phrases_df.apply(map_categories, axis=1)

key_phrases_category_count={}
for key, value in key_phrases_category.items():
    key_phrases_category_count[key] = len(value)

from collections import OrderedDict
key_phrases_category_count = OrderedDict(
                               sorted(key_phrases_category_count.items(), 
                                      key=lambda kv: kv[1], 
                                      reverse=True)
                            )

print(len(key_phrases_category_count))
# print(key_phrases_category_count)

In [ ]:
key_phrases_category_count

In [ ]:
key_phrases_df.id = key_phrases_df.id.astype(int)
key_phrases_df.keyPhrases = key_phrases_df.keyPhrases.astype(str)
key_phrases_df.shortKeyPhrases = key_phrases_df.shortKeyPhrases.astype(str)

metadata_keyPhrases = pd.merge(
    key_phrases_df[["id", "keyPhrases", "shortKeyPhrases"]], 
    sample_tweets[["id","text", "Posted time", "Bio location"]], 
    on="id", 
    how="inner")

metadata_keyPhrases.sort_values(by=["keyPhrases", "Bio location"], inplace=True)

In [ ]:
metadata_keyPhrases[["keyPhrases", "text"]].head(100)

In [ ]:
metadata_keyPhrases.to_csv("tweet_keyPhrases.csv")

__________________________________________________________________________________________________________________